# Playing Time Influenced by Gender as a Predictor for Game Newsletter Subscription

**Question 1 (General):** What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

**Specific Interpretation:** What is the relationship between total played hours and subscription class of a player? How does a player’s gender strengthen or weaken the predictive ability of average session time for subscription class?

To answer this question, we use the following variables:
1. `played hours`: the number of hours a player has logged on the server.
2. `subscribe`: a binary classification that flags whether a player subscribes to a game-related newsletter.
3. `gender`: a qualitative variable. For simplicity, we use a binary flag where one flag is male and the other is gender-diverse, which is an umbrella term that we will apply here to include women, non-binary people, two-spirit people, agender people, and those who responded 'other' in the survey. We will exclude those who answer 'prefer not to say'.

To explore the impacts of gender on the predictive ability of `played hours`, we will start by training a classifier including data from all players. `played_hours` will be used as the predictive variable for the binary classification `subscribe` in a standard knn classification scheme. Then, we will split the data into two groups: one comprised of only male players, and one comprised of only gender diverse players. We will train one new classifier on each. This will leave us with three classifiers which we can then directly compare the skill of using standard classification metrics. 

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
players <- read_csv('https://raw.githubusercontent.com/kathleenramsey/dsci100_group23/main/Project%20Planning%20Players.csv')
players